In [7]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter



In [8]:
paths = [
    "files/apostila.pdf",
    "files/LLM.pdf",
]

pages = []

## Cria uma lista com todos os documentos
for path in paths:
    loader = PyPDFLoader(path)
    docs = loader.load()
    pages.extend(docs)

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50, separators=["\n\n", "\n", ". ", " ", ""])

docs = splitter.split_documents(pages)

normalized_docs = []

for doc in docs:
    # Remove repeated dots, dashes and spaces
    cleaned_text = doc.page_content
    cleaned_text = " ".join(cleaned_text.split())  # Remove extra whitespace
    cleaned_text = cleaned_text.replace("....", "")
    cleaned_text = cleaned_text.replace("...", "")
    cleaned_text = cleaned_text.replace("..", "")
    cleaned_text = cleaned_text.replace("----", "-")
    cleaned_text = cleaned_text.replace("---", "-")
    cleaned_text = cleaned_text.replace("--", "-")
    cleaned_text = cleaned_text.replace("  ", "")
    cleaned_text = cleaned_text.strip()

    # Create new document with cleaned text
    doc.page_content = cleaned_text
    normalized_docs.append(doc)

print(len(docs))

Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 50 0 (offset 0)
Ignoring wrong pointing object 52 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)
Ignoring wrong pointing object 56 0 (offset 0)
Ignoring wrong pointing object 58 0 (offset 0)
Ignoring wrong pointing object 70 0 (offset 0)
Ignoring wrong pointing object 72 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 91 0 (offset 0)
Ignoring wrong pointing object 103 0 (offset 0)
Ignoring wrong pointing object 108 0 (offset 0)
Ignoring wrong pointing object 149 0 (offset 0)
Ignoring wrong pointing object 155 0 (offset 0)
Ignoring wrong pointing object 158 0 (offset 0)
Ignoring wrong pointing object 160 0 (offset 0)
Ignoring wrong pointing object 163 0 (offset 0)
Ignori

146


In [15]:
for index, doc in enumerate(docs):
    doc.metadata["source"] = paths[index % len(paths)]
    doc.metadata['doc_id'] = index

print(docs[1])



page_content='Sumário PREFÁCIO 1 1. INTRODUÇÃO 2 1.1 Característi cas da linguagem Python . 2' metadata={'source': 'files/LLM.pdf', 'page': 1, 'doc_id': 1}


In [19]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

## Banco vetorial

from langchain_chroma import Chroma


dir = "db/chroma_db_retrieval"

vector_store = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory=dir,
)

### Sematinc Search

In [10]:
ask = "O que é um LLM?"

docs = vector_store.similarity_search(ask, k=3)

for doc in docs:
    print(doc.page_content)
    print(doc.metadata)
    print("-" * 100)


. Se você quiser se aprofundar um pouco mais e expandir seus conhecimentos e compreensão dos fundamentos dos LLMs, recomendamos conferir nosso curso sobre LLMs. Você aprenderá como desenvolver aplicativos prontos para produção com LLMs e se aprofundará na teoria por trás dos modelos de fundação
{'doc_id': 141, 'page': 7, 'source': 'files/LLM.pdf'}
----------------------------------------------------------------------------------------------------
. Se você quiser se aprofundar um pouco mais e expandir seus conhecimentos e compreensão dos fundamentos dos LLMs, recomendamos conferir nosso curso sobre LLMs. Você aprenderá como desenvolver aplicativos prontos para produção com LLMs e se aprofundará na teoria por trás dos modelos de fundação
{'page': 7, 'source': 'files/LLM.pdf'}
----------------------------------------------------------------------------------------------------
PARTE 1 Introdução Definição de LLM (tradução livre: grandes modelos de linguagem) LLMs são sistemas de IA desenv

### Max Margina Relevance

In [21]:
docs = vector_store.max_marginal_relevance_search(ask, k=3, fetch_k=10)

for doc in docs:
    print(doc.page_content)
    print(doc.metadata)
    print("-" * 100)

. Se você quiser se aprofundar um pouco mais e expandir seus conhecimentos e compreensão dos fundamentos dos LLMs, recomendamos conferir nosso curso sobre LLMs. Você aprenderá como desenvolver aplicativos prontos para produção com LLMs e se aprofundará na teoria por trás dos modelos de fundação
{'doc_id': 141, 'page': 7, 'source': 'files/LLM.pdf'}
----------------------------------------------------------------------------------------------------
PARTE 1 Introdução Definição de LLM (tradução livre: grandes modelos de linguagem) LLMs são sistemas de IA desenvolvidos para processar e analisar enormes quantidades de dados de linguagem natural e, em seguida, usar essas informações para gerar respostas às solicitações dos usuários
{'doc_id': 101, 'page': 1, 'source': 'files/LLM.pdf'}
----------------------------------------------------------------------------------------------------
. Conclusão e diretrizes gerais Em última análise, cada organização terá desafios únicos a superar, e não exi

### Filter


In [27]:
ask = "O que a apostila de LLM fala sobre o OpenAI e sobre o ChatGPT?"

docs = vector_store.similarity_search(ask, k=10, filter={"source": "files/LLM.pdf"})

for doc in docs:
    print(doc.page_content)
    print(doc.metadata)

. 2023 Os LLMs de código aberto começam a apresentar resultados cada vez mais impressionantes, com lançamentos como Dolly 2.0, LLaMA, Alpaca e Vicuna. O GPT-4 também é lançado, estabelecendo um novo referencial tanto em tamanho de parâmetros quanto em desempenho.
{'doc_id': 109, 'page': 2, 'source': 'files/LLM.pdf'}
. Comunidades como a Hugging Face reúnem centenas de milhares de modelos de contribuidores que podem ajudar a resolver muitos casos de uso específicos, como geração de texto, resumo e classificação. A comunidade de código aberto está rapidamente alcançando o desempenho dos modelos proprietários, mas ainda não conseguiu igualar o desempenho de algo como o GPT-4.
{'doc_id': 131, 'page': 5, 'source': 'files/LLM.pdf'}
E-BOOK Um guia compacto sobre Large Language Models (LLM)
{'doc_id': 99, 'page': 0, 'source': 'files/LLM.pdf'}
. 2012 Os avanços em arquiteturas de deep learning e conjuntos de dados maiores levaram ao desenvolvimento do GPT (Transformadores Pré-treinados Generati

### LLM Aided Retrieval = Melhora a busca dentro do RAG

### Tive que fazer "manualmente pq o Chroma n tem suport para SelfQueryRetriever mostrado


In [3]:
from langchain_openai import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.schema import AttributeInfo


In [4]:
metadata_info = [
    AttributeInfo(
        name="source",
        description="Nome da apostila de onde o texto foi extraido. Pode ser 'apostila.pdf' ou 'LLM.pdf'",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="A página da apostila de onde o texto foi extraido. Número inteiro",
        type="integer",
    ),
]

In [22]:
from langchain_openai import ChatOpenAI
import json
import re

# 1. Instancia o LLM
llm = ChatOpenAI(model="gpt-4o")

# 2. Descrição dos metadados (AttributeInfo que você já tem)
metadata_info = [
    {
        "name": "source",
        "description": "Nome da apostila de onde o texto foi extraído. Pode ser 'apostila.pdf' ou 'LLM.pdf'",
        "type": "string",
    },
    {
        "name": "page",
        "description": "A página da apostila de onde o texto foi extraído. Número inteiro.",
        "type": "integer",
    },
]

# 3. Gera descrição dos atributos
attribute_descriptions = "\n".join(
    f"- {attr['name']}: {attr['description']} (tipo: {attr['type']})"
    for attr in metadata_info
)


# 4. Função para criar o prompt
def create_prompt(user_query):
    return f"""
Você é um assistente que ajuda a melhorar consultas em uma base de dados vetorial.
Aqui estão os metadados disponíveis:
{attribute_descriptions}

A consulta do usuário é:
\"{user_query}\"

Tarefa:
1. Reescreva a consulta para melhorar a busca.
2. Sugira um filtro baseado nos metadados disponíveis (em formato JSON).

Responda no formato:
Query Reformulada: <texto>
Filtro: <JSON>
"""


# 5. Pergunta do usuário
user_query = "O que a apostila de LLM fala sobre o OpenAI e sobre o ChatGPT?"

# 6. Gera o prompt e faz o LLM gerar a reformulação
prompt = create_prompt(user_query)
response = llm.invoke(prompt)

# 7. Parseia a resposta
print(response.content)

reformulated_query = re.search(r"Query Reformulada:\s*(.*)", response.content).group(1)

# Novo regex para pegar o JSON dentro de blocos ```
filter_match = re.search(r"```json\s*(\{.*\})\s*```", response.content, re.DOTALL)

if filter_match:
    filter_json = filter_match.group(1)
    filter_dict = json.loads(filter_json)
else:
    print("Erro: Não foi possível extrair o filtro.")
    filter_dict = {}

print("Query reformulada:", reformulated_query)
print("Filtro sugerido:", filter_dict)

# 8. Faz a busca no seu Chroma vector_store
docs = vector_store.similarity_search(query=reformulated_query, k=5, filter=filter_dict)


# 9. Exibe resultados
for doc in docs:
    print(doc.page_content)
    print(doc.metadata)

Query Reformulada: "Conteúdo sobre OpenAI e ChatGPT na apostila LLM"

Filtro: 
```json
{
  "source": "LLM.pdf"
}
```
Query reformulada: "Conteúdo sobre OpenAI e ChatGPT na apostila LLM"
Filtro sugerido: {'source': 'LLM.pdf'}
